In [ ]:
# first we define relevant directories
import sys
import os
import copy
# project directory
project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# src directory 
src_dir = os.path.join(project_dir, 'src')
sys.path.insert(0, src_dir)
print("src_dir ", src_dir)

In [ ]:
from train_eval import *

In [ ]:
torch.__version__

In [ ]:
NUM_PROCESSORS = mp.cpu_count() - 2

In [ ]:
# check the available devices
print(torch.cuda.device_count())
print(torch.cuda.current_device())

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '' # specify the visibile devices before running the script

In [ ]:
def train_eval_models(bestmodels_dir, wrk_dir, model_class, y_codebook, num_epochs, 
                      to_gpu, memmap, dataset_dir, func_mode = 'train', options={}):
    
    if(func_mode == 'train'):
        target_folder = 'best_model'
        dsettypes = ['train', 'validation']
    elif(func_mode == 'validation'):
        target_folder = None
        dsettypes = ['validation']
    else:
        target_folder = None
        dsettypes = ['test']
    
    bestmodels_lst = retrieve_bestmodels(bestmodels_dir, target_folder)
    folds_dir = ["fold_{}_none".format(i) for i in range(5)]
            
    classweight_option = options.get('classweight_option')
    if(model_class == 'RNN_Labeler'):
        model_train = rnn_run
    elif(model_class ==  'RNNSS_Labeler'):
        model_train = rnnss_run
    # to enforce what options are available    
    elif(model_class in {'CRF_Labeler', 'CRF_Pair_Labeler', 'RNNCRF_Pair_Labeler', 'RNNCRF_Unary_Labeler'}): 
        stopstate_symb = options.get('stopstate_symb')
        if(stopstate_symb):
            y_codebook[stopstate_symb] = len(y_codebook)
        model_train = rnncrf_run
    elif(model_class in {'NN_Labeler', 'CNN_Labeler', 'CNNWide_Labeler'}):
        model_train = cnn_nn_run
        
    # prepare the arguments for function call
    rank = 0
    args = []
    evaldirs = []
    for bestmodel in bestmodels_lst:
        bestmodel_config = ReaderWriter.read_data(bestmodel.model_config_pth)
        model_statedict_pth = bestmodel.model_statedict_pth
        model_prefix = bestmodel.model_prefix
        for fold in folds_dir:
            print("current_fold: ", fold)
            if(func_mode == 'test'):
                # case of the fold not matching
                if(len(model_prefix.split(fold)) == 1):
                    continue
                else:
                    mweights_path = model_statedict_pth
            elif(func_mode == 'train'):
                mweights_path = None
            elif(func_mode == 'validation'):
                mweights_path = model_statedict_pth

            # make a copy of options
            options_c = copy.deepcopy(options)
            target_dsets, input_dim = load_dataset(os.path.join(dataset_dir, fold, 'dataset_tuple.pkl'), dsettypes, memmap)
            if(classweight_option):
                class_weights = ReaderWriter.read_data(os.path.join(dataset_dir, 
                                                                    fold, 
                                                                    'train_pdtm', 
                                                                    'classweights_'+classweight_option+'.pkl'))
                options_c['class_weights'] = class_weights        
            eval_dir = create_directory('{}_{}'.format(bestmodel.model_prefix, fold), wrk_dir)
            evaldirs.append(eval_dir)
            # add the fold_id option
            options_c['fold_id'] = fold
            args.append((bestmodel_config, model_class, target_dsets, y_codebook, 
                         num_epochs, eval_dir, rank, to_gpu, memmap, model_statedict_pth, options_c))

            rank +=1
    
    # multiprocessing the model runs
    outlog = os.path.join(wrk_dir, 'out.log')

    num_models = len(args)
    num_processors = mp.cpu_count()-1

    if(num_processors > NUM_PROCESSORS):
        num_processors = NUM_PROCESSORS
    if(num_models < num_processors):
        num_processors = num_models
    print("Number of models to be evaluated: ", num_models)

    processes = []
    counter = 0
    tick = datetime.datetime.now()
    while len(args):
        if(len(args) < num_processors):
            num_processors = len(args)
        for __ in range(num_processors):
            arg = args.pop()
            p = mp.Process(target=model_train, args=arg)
            p.start()
            processes.append(p)
            print("counter ", counter)
            counter+=1
        for p in processes:
            p.join()
            print("we are joining {}".format(p))
            
    tock = datetime.datetime.now()
    line = "total number of seconds: {}\n".format((tock-tick).seconds)
    ReaderWriter.write_log(line, outlog, mode='a')
    if(func_mode == 'train'):
        dtype = 'validation'
    elif(func_mode == 'test'):
        dtype = 'test'
    elif(func_mode == 'validation'):
        dtype = 'test'
    scores = get_perfscores(evaldirs, dtype, outlog, None)
    return(scores)

In [ ]:
decoded_dir = create_directory('decoded_output', project_dir)
hyperparam_optimzmodel_dir =  create_directory('hyperparam_optimz_models', project_dir)
dataset_dir = os.path.join(project_dir, 'dataset_neural')

#### Separate each model run/test in a cell -- this can be easily optmized to run all models at once!!

## RNN/RNNSS train/test

In [ ]:
# checking __main__ condition helps preventing "weird" issues when running multiprocesses on Windows OS!!
if __name__ == "__main__":
    to_gpu = True # run on GPU
    memmap = True # data is memory mapped originally using numpy memmap
    bidirection = False
    classweight_option = 'last_indx'
    loss_modes = ('Convex_HF_LastHF', 'LastHF', 'Uniform_HF', 'Convex_HF_NonHF')
    rgrad_mode = 'clip_norm'
    rgrad_limit = (None, 1)
    model_classes = ('RNN_Labeler', 'RNNSS_Labeler')
    prefixes = ('rnn', 'rnnss')
    y_codebooks = ({0:0, 1:1}, {0:0, 1:1, '__START__':2})
    
    # train/validation phase
    num_epochs = 2
    for i, model_class in enumerate(model_classes):
        y_codebook = y_codebooks[i]
        prefix = prefixes[i]
        # where to find the model with *best* hyperparams      
        target_dir = create_directory(prefix, hyperparam_optimzmodel_dir)
        trainmodel_dir = os.path.join(project_dir, 'train_models', prefix)
        for loss_mode in loss_modes:
            fname = "{}_{}".format(loss_mode, rgrad_mode+str(rgrad_limit[-1]))
            wrk_dir = create_directory('{}_{}_train'.format(prefix, fname), trainmodel_dir)
            scoredict = train_eval_models(target_dir, wrk_dir, model_class, y_codebook, num_epochs,
                                          to_gpu, memmap, dataset_dir,'train',
                                         {'classweight_option':classweight_option,
                                          'reg_type':'l2',
                                          'loss_mode':loss_mode,
                                          'bidirection':bidirection,
                                          'rgrad_mode':rgrad_mode,
                                          'rgrad_limit':rgrad_limit})
            print("training:")
            print(scoredict)
            print("-"*25)
            
    # test phase
    num_epochs = 1
    for i, model_class in enumerate(model_classes):
        y_codebook = y_codebooks[i]
        prefix = prefixes[i]
        trainmodel_dir = os.path.join(project_dir, 'train_models', prefix)
        testmodel_dir = os.path.join(project_dir, 'test_models', prefix)
        for loss_mode in loss_modes:
            fname = "{}_{}".format(loss_mode, rgrad_mode+str(rgrad_limit[-1]))
            target_dir = create_directory('{}_{}_train'.format(prefix, fname), trainmodel_dir)
            wrk_dir = create_directory('{}_{}_test'.format(prefix, fname), testmodel_dir)
            scoredict = train_eval_models(target_dir, wrk_dir, model_class, y_codebook, num_epochs,
                                          to_gpu, memmap, dataset_dir,'test',
                                         {'classweight_option':classweight_option,
                                          'reg_type':'l2',
                                          'loss_mode':loss_mode,
                                          'bidirection':bidirection,
                                          'dec_outdir':create_directory('{}_lossmode_{}_decoded'.format(prefix, loss_mode), decoded_dir)})
            print("testing:")
            print(scoredict)
            print("-"*25)


## CNN/CNNWide/MLP train/test

In [ ]:
if __name__ == "__main__":    
    to_gpu = True
    memmap = True
    classweight_option = 'last_indx'
    y_codebook = {0:0,1:1}
    rgrad_mode = 'clip_norm'
    rgrad_limit = (None, 1)
    model_classes = ('CNN_Labeler', 'CNNWide_Labeler', 'NN_Labeler')
    prefixes = ('cnn', 'cnnwide', 'nn')
    # train/validation phase
    num_epochs = 2
    for i, model_class in enumerate(model_classes):
        prefix = prefixes[i]
        target_dir = create_directory(prefix, hyperparam_optimzmodel_dir)
        trainmodel_dir = os.path.join(project_dir, 'train_models', prefix)
        wrk_dir = create_directory('{}_train'.format(prefix), trainmodel_dir)
        scoredict = train_eval_models(target_dir, wrk_dir, model_class, y_codebook, num_epochs,
                                      to_gpu, memmap, dataset_dir,'train',
                                     {'classweight_option':classweight_option,
                                      'reg_type':'l2',
                                      'rgrad_mode':rgrad_mode,
                                      'rgrad_limit':rgrad_limit})
        print("training:")
        print(scoredict)
        print("-"*25)
    # test phase
    num_epochs = 1
    for i, model_class in enumerate(model_classes):
        prefix = prefixes[i]
        trainmodel_dir = os.path.join(project_dir, 'train_models', prefix)
        target_dir = create_directory('{}_train'.format(prefix), trainmodel_dir)
        testmodel_dir = os.path.join(project_dir, 'test_models', prefix)
        wrk_dir = create_directory('{}_test'.format(prefix), testmodel_dir)
        scoredict = train_eval_models(target_dir, wrk_dir, model_class, y_codebook, num_epochs,
                                      to_gpu, memmap, dataset_dir, 'test',
                                      {'classweight_option':classweight_option,
                                       'dec_outdir':create_directory('{}_decoded'.format(prefix), decoded_dir)}) 
        print("testing:")
        print(scoredict)
        print("-"*25)

## RNNCRF {Pair, Unary} train/test

In [ ]:
if __name__ == "__main__":
    to_gpu = True
    memmap = True
    bidirection = False
    y_codebook = {0:0, 1:1, '__START__':2}
    decoder_type = 'viterbi'
    rgrad_mode = 'clip_norm'
    rgrad_limit = (None, 1)
    model_classes = ('RNNCRF_Pair_Labeler', 'RNNCRF_Unary_Labeler')
    prefixes = ('rnncrfpair', 'rnncrfunary')
    # train/validation phase
    num_epochs = 2
    for i, model_class in enumerate(model_classes):
        prefix = prefixes[i]
        target_dir = create_directory(prefix, hyperparam_optimzmodel_dir)
        trainmodel_dir = os.path.join(project_dir, 'train_models', prefix)
        wrk_dir = create_directory('{}_train'.format(prefix), trainmodel_dir)
        scoredict = train_eval_models(target_dir, wrk_dir, model_class, y_codebook, num_epochs,
                                      to_gpu, memmap, dataset_dir,'train',
                                     {'reg_type':'l2',
                                      'decoder_type':decoder_type,
                                      'bidirection':bidirection,
                                      'rgrad_mode':rgrad_mode,
                                      'rgrad_limit':rgrad_limit})
        print("training:")
        print(scoredict)
        print("-"*25)
    # test phase
    num_epochs = 1
    for i, model_class in enumerate(model_classes):
        prefix = prefixes[i]
        trainmodel_dir = os.path.join(project_dir, 'train_models', prefix)
        target_dir = create_directory('{}_train'.format(prefix), trainmodel_dir)
        testmodel_dir = os.path.join(project_dir, 'test_models', prefix)
        wrk_dir = create_directory('{}_test'.format(prefix), testmodel_dir)
        scoredict = train_eval_models(target_dir, wrk_dir, model_class, y_codebook, num_epochs,
                                      to_gpu, memmap, dataset_dir,'test',
                                     {'reg_type':'l2',
                                      'decoder_type':decoder_type,
                                      'bidirection':bidirection,
                                      'dec_outdir':create_directory('{}_decoded'.format(prefix), decoded_dir)})
        print("testing:")
        print(scoredict)
        print("-"*25)


## CRF/Neural CRF {Pair, Unary} train/test

In [ ]:
if __name__ == "__main__":
    to_gpu = True
    memmap = True
    y_codebook = {0:0, 1:1, '__START__':2}
    decoder_type = 'viterbi'
    rgrad_mode = 'clip_norm'
    rgrad_limit = (None, 1)
    model_classes = ('CRF_Pair_Labeler', 'CRF_Labeler')
    crf_types = ('nn','only')
    # train/validation phase
    num_epochs = 2
    for model_class in model_classes:
        for crf_type in crf_types:
            if('Pair' in model_class):
                prefix = 'crf{}pair'.format(crf_type)
            else:
                prefix = 'crf{}'.format(crf_type)
            target_dir = create_directory(prefix, hyperparam_optimzmodel_dir)
            trainmodel_dir = os.path.join(project_dir, 'train_models', prefix)
            wrk_dir = create_directory('{}_train'.format(prefix), trainmodel_dir)
            scoredict = train_eval_models(target_dir, wrk_dir, model_class, y_codebook, num_epochs,
                                          to_gpu, memmap, dataset_dir,'train',
                                         {'reg_type':'l2',
                                          'decoder_type':decoder_type,
                                          'crf_type': '{}_crf'.format(crf_type),
                                          'bidirection':bidirection,
                                          'rgrad_mode':rgrad_mode,
                                          'rgrad_limit':rgrad_limit})
            print("training:")
            print(scoredict)
            print("-"*25)
    # test phase
    num_epochs = 1
    for model_class in model_classes:
        for crf_type in crf_types:
            if('Pair' in model_class):
                prefix = 'crf{}pair'.format(crf_type)
            else:
                prefix = 'crf{}'.format(crf_type)
            trainmodel_dir = os.path.join(project_dir, 'train_models', prefix)
            target_dir = create_directory('{}_train'.format(prefix), trainmodel_dir)
            testmodel_dir = os.path.join(project_dir, 'test_models', prefix)
            wrk_dir = create_directory('{}_test'.format(prefix), testmodel_dir)
            scoredict = train_eval_models(target_dir, wrk_dir, model_class, y_codebook, num_epochs,
                                          to_gpu, memmap, dataset_dir,'test',
                                         {'reg_type':'l2',
                                          'decoder_type':decoder_type,
                                          'crf_type': '{}_crf'.format(crf_type),
                                          'bidirection':bidirection,
                                          'dec_outdir':create_directory('{}_decoded'.format(prefix), decoded_dir)})
            print("testing:")
            print(scoredict)
            print("-"*25)